# Advent of Code 2021 - Day 16

## File Input

In [160]:
input = open('./input.txt', "r")
transmission = input.read()
input.close()

test_strings = ["D2FE28", "38006F45291200", "EE00D40C823060", "8A004A801A8002F478", "620080001611562C8802118E34", "C0015000016115A2E0802F182340", "A0016C880162017C3686B18A3D4780"]

## Part 1

In [478]:
hex_to_bin = {
    "0": "0000",
    "1": "0001",
    "2": "0010",
    "3": "0011",
    "4": "0100",
    "5": "0101",
    "6": "0110",
    "7": "0111",
    "8": "1000",
    "9": "1001",
    "A": "1010",
    "B": "1011",
    "C": "1100",
    "D": "1101",
    "E": "1110",
    "F": "1111"
}

def convert_hex_to_binary(hex_format):
    return "".join([hex_to_bin[char] for char in hex_format])

def get_sub_packet(binary_transmission):
    packet_version = int(binary_transmission[:3], 2)
    packet_type_id = int(binary_transmission[3:6], 2)
    is_literal_value = packet_type_id == 4
    pointer = 6
        
    if is_literal_value:        
        # skip over values of literal number
        while binary_transmission[pointer] == "1":
            pointer += 5
        pointer += 5
        
        return binary_transmission[:pointer], pointer
    else:
        length_type_id = binary_transmission[pointer]
        pointer += 1
                        
        if length_type_id == "0":  # length is 15-bit number representing number of bits in sub-packets
            sub_packet_length = int(binary_transmission[pointer:pointer+15], 2)
            pointer += 15
            
            return binary_transmission[:pointer+sub_packet_length], pointer+sub_packet_length
        else: # length is 11-bit number representing number of sub-packets
            num_sub_packets = int(binary_transmission[pointer:pointer+11], 2)
            pointer += 11
            
            subpacket = binary_transmission[:pointer]
            
            for _ in range(num_sub_packets):
                new_sub, new_idx = get_sub_packet(binary_transmission[pointer:])
                subpacket += new_sub
                pointer += new_idx
                
            return subpacket, pointer

def get_version_sum(binary_transmission):
    if len(binary_transmission) <= 7:
        return 0
    
    packet_version = int(binary_transmission[:3], 2)
    packet_type_id = int(binary_transmission[3:6], 2)
    is_literal_value = packet_type_id == 4
    
    version_sum = packet_version
    pointer = 6
        
    if is_literal_value:        
        literal_value = ""
        
        # skip over values of literal number
        while binary_transmission[pointer] == "1":
            literal_value += binary_transmission[pointer+1:pointer+5]
            pointer += 5
        literal_value += binary_transmission[pointer+1:pointer+5]
        pointer += 5
        
        literal_value = int(literal_value, 2)        
        print("literal ", f"version {version_sum}", f"literal value {literal_value}")
                
        return version_sum + get_version_sum(binary_transmission[pointer:])
    else:
        length_type_id = binary_transmission[pointer]
        pointer += 1
        
        print("operator", f"version {version_sum}", f"length type id {length_type_id}")
                
        if length_type_id == "0":  # length is 15-bit number representing number of bits in sub-packets
            sub_packet_length = int(binary_transmission[pointer:pointer+15], 2)
            pointer += 15
                        
            # now need to get version sum for all subpackets, which will take up the next sub_packet_length bits
            return version_sum + get_version_sum(binary_transmission[pointer:pointer+sub_packet_length]) + get_version_sum(binary_transmission[pointer+sub_packet_length:])

        else: # length is 11-bit number representing number of sub-packets
            num_sub_packets = int(binary_transmission[pointer:pointer+11], 2)
            pointer += 11
                                                
            for _ in range(num_sub_packets):
                subpacket, sub_pointer = get_sub_packet(binary_transmission[pointer:])
                version_sum += get_version_sum(subpacket)
                pointer += sub_pointer
                
            return version_sum + get_version_sum(binary_transmission[pointer:])

get_version_sum(convert_hex_to_binary(transmission))

operator version 7 length type id 1
operator version 0 length type id 1
literal  version 5 literal value 1161
operator version 3 length type id 1
operator version 5 length type id 0
literal  version 7 literal value 2
literal  version 6 literal value 4
literal  version 7 literal value 13
operator version 3 length type id 1
literal  version 2 literal value 6
literal  version 5 literal value 6
literal  version 4 literal value 3
operator version 5 length type id 1
literal  version 3 literal value 5250146
operator version 7 length type id 1
literal  version 5 literal value 246310804
literal  version 1 literal value 246310804
operator version 7 length type id 0
operator version 6 length type id 1
literal  version 4 literal value 4401420
literal  version 4 literal value 55853
literal  version 1 literal value 6
literal  version 4 literal value 3759
operator version 4 length type id 1
operator version 2 length type id 1
operator version 0 length type id 1
operator version 0 length type id 1
ope

984

## Part 2

In [494]:
from math import prod

def parse_packet(binary_transmission):
    if len(binary_transmission) <= 7:
        return []
    
    packet_version = int(binary_transmission[:3], 2)
    packet_type_id = int(binary_transmission[3:6], 2)
    is_literal_value = packet_type_id == 4
    
    version_sum = packet_version
    pointer = 6
        
    if is_literal_value:        
        literal_value = ""
        
        # skip over values of literal number
        while binary_transmission[pointer] == "1":
            literal_value += binary_transmission[pointer+1:pointer+5]
            pointer += 5
        literal_value += binary_transmission[pointer+1:pointer+5]
        pointer += 5
        
        literal_value = int(literal_value, 2)        
        print("literal ", f"version {version_sum}", f"literal value {literal_value}")
                
        return [literal_value] + parse_packet(binary_transmission[pointer:])
    else:
        length_type_id = binary_transmission[pointer]
        pointer += 1
        
        print("operator", f"version {version_sum}", f"length type id {length_type_id}")
        
        subpackets = []
                
        if length_type_id == "0":  # length is 15-bit number representing number of bits in sub-packets
            sub_packet_length = int(binary_transmission[pointer:pointer+15], 2)
            pointer += 15
                                                            
            subpackets += parse_packet(binary_transmission[pointer:pointer+sub_packet_length]) + [parse_packet(binary_transmission[pointer+sub_packet_length:])]
        else: # length is 11-bit number representing number of sub-packets
            num_sub_packets = int(binary_transmission[pointer:pointer+11], 2)
            pointer += 11
                                                
            for _ in range(num_sub_packets):
                subpacket, sub_pointer = get_sub_packet(binary_transmission[pointer:])
                subpackets += parse_packet(subpacket)
                pointer += sub_pointer
                
            subpackets += [parse_packet(binary_transmission[pointer:])]
            
        midpoint = 0
        
        for i, x in enumerate(subpackets):
            if not isinstance(x, int):
                midpoint = i
                break
                
        int_values = subpackets[:midpoint]
        padded_value = subpackets[midpoint]
        
        return_val = None
                                                
        if packet_type_id == 0:
            return_val = [sum(int_values)]
        elif packet_type_id == 1:
            return_val = [prod(int_values)]
        elif packet_type_id == 2:
            return_val = [min(int_values)]
        elif packet_type_id == 3:
            return_val = [max(int_values)]
        elif packet_type_id == 5:
            return_val = [1] if subpackets[0] > subpackets[1] else [0]
        elif packet_type_id == 6:
            return_val = [1] if subpackets[0] < subpackets[1] else [0]
        elif packet_type_id == 7:
            return_val = [1] if subpackets[0] == subpackets[1] else [0]
        
        return return_val + padded_value

parse_packet(convert_hex_to_binary(transmission))

[1015320896946]